# Web Scraping  Selenium
### LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

### Import Selenium and WebDriver Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

url='https://www.latamairlines.com/py/es/ofertas-vuelos?origin=ASU&inbound=null&outbound=2023-02-24T15%3A00%3A00.000Z&destination=MAD&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED_A'

### Import Explicit Wait libraries

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

### Function to obtain flight prices

In [3]:
def obtener_precios(vuelo):
    '''
    Funcion que retorna una lista de diccionarios con las distintas tarifas del vuelo
    '''
    tarifas = vuelo.find_elements('xpath','.//ol[@class="sc-buGlAa jhwXGF"]/li')
    precios=[]

    for tarifa in tarifas:
        nombre=tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[1]/span[@class="sc-fhiYOA iwcbaW"]').text
        moneda=tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[contains(@class,"currency")]').text
        valor=tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[@class="sc-ckYZGd grNCid"]').text

        dict_tarifa={nombre:{'moneda':moneda,'valor':valor}}

        precios.append(dict_tarifa)
    return precios

### Function to obtain scale data

In [4]:
def obtener_datos_escalas(vuelo):
    '''
    Funcion que retorno una lista dédiccionarios con la información de las escalas de cada vuelo
    '''
    segmentos= vuelo.find_elements('xpath','//article[@class="sc-lffWgi vYKpy"]//section[@class="sc-fGSyRc fCuylQ"]')
    datos_escalas=[]
    duracion_escalas_dict={}

    for segmento in segmentos:
        wait_segmento= WebDriverWait(segmento, 15)

        #Origen
        origen_segmento= wait_segmento.until(EC.presence_of_element_located((By.XPATH,'.//div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[last()-1]'))).text

        #Hora de Salida
        hora_salida= wait_segmento.until(EC.presence_of_element_located((By.XPATH,'.//div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[2]'))).text

        #Destino Segmento
        destino_segmento= wait_segmento.until(EC.presence_of_element_located((By.XPATH,'.//div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[1]'))).text

        # hora de llegada
        hora_llegada= wait_segmento.until(EC.presence_of_element_located((By.XPATH,'.//div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[2]'))).text

        #duracion del vuelo
        duracion_segmento= wait_segmento.until(EC.presence_of_element_located((By.XPATH,'.//div[@class="sc-ewMkZo hQNSAX"]/span[2]'))).text

        #Numero de vuelo
        numero_vuelo_segmento=wait_segmento.until(EC.presence_of_element_located((By.XPATH,'.//div[@class="sc-dzQEYZ dslPlz airline-wrapper"]'))).text

        #modelo del avion
        modelo_avion_segmento=wait_segmento.until(EC.presence_of_element_located((By.XPATH,'.//div[@class="sc-sVRsr eXYUTi"]//span[@class="airplane-code"]'))).text

        datos_escalas_dict ={
            'origen':origen_segmento,
            'hora_salida':hora_salida,
            'destino':destino_segmento,
            'hora_llegada':hora_llegada,
            'duracion':duracion_segmento,
            'numero_vuelo':numero_vuelo_segmento,
            'modelo_avion':modelo_avion_segmento
        }
        datos_escalas.append(datos_escalas_dict)
    
    #Escalas
    escalas_vuelo= vuelo.find_elements('xpath','//section[@class="sc-kiXyGy sc-eZXMBi dKgCnQ connectionInfo"]')

    for num_escla, escala in enumerate(escalas_vuelo):
        escala_vuelo= escala.find_element('xpath','.//div[@class="sc-ekQYnd cByWfv"]//span[@class="connection-text"]').text
        duracion_escala_vuelo=escala.find_element('xpath','.//div[@class="sc-ekQYnd cByWfv"]//span[@class="time"]').text
        duracion_escalas={
            f'escala_{num_escla+1}':escala_vuelo,
            f'duracion_escala_{num_escla+1}':duracion_escala_vuelo
        }
        duracion_escalas_dict.update(duracion_escalas)
    datos_escalas.append({'duracion_escalas':duracion_escalas_dict})

    return datos_escalas





### Function to obtain overall times

In [5]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada vuelo, incluyendo la duracion.
    Nota: la duración del vuelo no es la hora de llegada menos la hora de salida
    '''

    #hora de salida
    hora_salida_vuelo=vuelo.find_element('xpath','.//div[@class="sc-klSiHT hjzFuR flight-information"]/span[1]').text

    #hora de llegada
    hora_llegada_vuelo=vuelo.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')

    #duracion del vuelo
    duracion_vuelo=vuelo.find_element('xpath','.//div[2]/span[2]').text

    tiempos_vuelo_dict={
        'hora_salida':hora_salida_vuelo,
        'hora_llegada':hora_llegada_vuelo,
        'duracion_vuelo':duracion_vuelo
    }

    return tiempos_vuelo_dict



### Main function that calls the other functions

In [6]:
def obtener_info(driver):
    '''
    Función principal que se encarga de llamar a las demas funciones para recolectar datos de los vuelos
    '''

    wait=WebDriverWait(driver, 10)
    vuelos=wait.until(EC.presence_of_all_elements_located((By.XPATH,'//ol/li[@class="sc-bvTASY cfqKKq"]')))

    print(f'Se encontraron {len(vuelos)} vuelos')
    print(f'Iniciando Scraping...')
    count=0
    info=[]
    
    for vuelo in vuelos:
        count+=1
        print(f'Analizando {count} de {len(vuelos)} vuelos')

        #Tiempos generales del vuelo
        tiempos= obtener_tiempos(vuelo)

        #Modal de vuelos internos o escalas
        WebDriverWait(vuelo,10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-iKiVwC fbWfQZ"]//a'))).click()

        element= vuelo.find_element('xpath','//div[@class="MuiDialogContent-root sc-bZQynM hnBdls"]')
        vertical_ordinate=100
        for i in range(0, 10):
            driver.execute_script("arguments[0].scrollTop = arguments[1]", element, vertical_ordinate)
            vertical_ordinate += 100
            time.sleep(1)

        print('Termino de cargar las escalas del vuelo')

        escalas=obtener_datos_escalas(vuelo)
        print(f'{escalas}\n')

        driver.find_element('xpath','//div[@class="MuiDialog-container MuiDialog-scrollPaper"]').click()

        vuelo.find_element('xpath','.//div[@class="sc-cIbcTr iuAYhf"]').click()

        WebDriverWait(driver,15).until(EC.presence_of_all_elements_located((By.XPATH,'.//ol[@class="sc-buGlAa jhwXGF"]/li')))

        print('La tabla de precios termino de cargar')

        precios=obtener_precios(vuelo)
        print(f'{precios}\n\n')

        vuelo.find_element('xpath','.//div[@id="undefined--button-wrapper"]/button').click()

        info.append({
            'precios':precios,
            'tiempos':tiempos,
            'escalas':escalas
        })
    
    return info



### We initialize driver and call the main function

In [7]:
options = Options()

options.add_argument('--incognito')

driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(url)

def close_dialog():
    '''
    Función que cierra los dialog o popups de la pagina web
    '''

    dialogs=driver.find_elements(By.XPATH,'//div[contains(@class,"MuiDialog-container")]')
    if len(dialogs):
        dialogs[0].find_element(By.XPATH,'.//button[contains(@class,"Dialog__CloseButton")]').click()

close_dialog()

delay=15

try:
    WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@id="WrapperBodyFlights"]//ol/li[@class="sc-bvTASY cfqKKq"]')))
    print('Los vuelos disponibles terminaron de cargar')
    obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')
finally:
    driver.close()

Los vuelos disponibles terminaron de cargar
Se encontraron 30 vuelos
Iniciando Scraping...
Analizando 1 de 30 vuelos
Termino de cargar las escalas del vuelo
[{'origen': 'ASU', 'hora_salida': '10:50 a. m.', 'destino': 'SCL', 'hora_llegada': '1:40 p. m.', 'duracion': '2 h 50 min', 'numero_vuelo': 'LA1324', 'modelo_avion': 'Airbus A320'}, {'origen': 'SCL', 'hora_salida': '5:10 p. m.', 'destino': 'GRU', 'hora_llegada': '8:55 p. m.', 'duracion': '3 h 45 min', 'numero_vuelo': 'LA702', 'modelo_avion': 'Boeing B787-8'}, {'origen': 'GRU', 'hora_salida': '11:30 p. m.', 'destino': 'MAD', 'hora_llegada': '1:25 p. m.', 'duracion': '9 h 55 min', 'numero_vuelo': 'LA8066', 'modelo_avion': 'Boeing B787-9'}, {'duracion_escalas': {'escala_1': 'Conexión Santiago de Chile', 'duracion_escala_1': '3 h 30 min', 'escala_2': 'Conexión Sao Paulo', 'duracion_escala_2': '2 h 35 min'}}]

La tabla de precios termino de cargar
[{'light': {'moneda': 'USD', 'valor': '799.80'}}, {'plus': {'moneda': 'USD', 'valor': '898.